## Bibliotecas

In [6]:
import pandas as pd                                     # Tratamiento de datos

from sklearn.model_selection import StratifiedKFold     # Clases desequelibradas
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
import statistics                                       # Estadisticas



## Leer dataset

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0
1,15810944,Male,35.0,20000.0,0
2,15668575,Female,26.0,43000.0,0
3,15603246,Female,27.0,57000.0,0
4,15804002,Male,19.0,76000.0,0


## Separar rasgos y clases 

In [3]:
x = dataset.iloc[:, [2, 3]].values
y = dataset.iloc[:, 4].values

## Modelo a evaluar

In [4]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

## Metodos de validación

#### Evaluar exactitud

In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(gnb, x, y, cv=10, scoring='accuracy')
print(f"[+]Exactitud: {100 * scores.mean()} %")
print(f"[+]Desviación estandar: {100 * scores.std()} %")


[+]Exactitud: 87.75 %
[+]Desviación estandar: 7.862092596758193 %


#### Validación cruzada

In [8]:
# --> Lista de metricas
lista_exactitud = []
lista_precison = []
lista_f1score = []
lista_sensibilidad = []

# --> Objeto para validacion
skf = StratifiedKFold(n_splits=10)


# ------------| Método de validación k-fold  cross-validation |------------
# --> Iteraciones
for entramiento, prueba in skf.split(x, y):
    # --> Separar datos en entrenamiento y prueba
    x_entrenamiento = [x[z] for z in entramiento]
    x_prueba = [x[z] for z in prueba]

    y_entrenamiento = [y[z] for z in entramiento]
    y_prueba = [y[z] for z in prueba]

    # --> Clasificador de bayes
    y_predicha = gnb.fit(x_entrenamiento, y_entrenamiento).predict(x_prueba)

    # --> Metricas
    lista_exactitud.append(accuracy_score(y_prueba, y_predicha))
    lista_precison.append(precision_score(y_prueba, y_predicha, average="macro"))
    lista_f1score.append(f1_score(y_prueba, y_predicha, average="macro"))
    lista_sensibilidad.append(recall_score(y_prueba, y_predicha, average="macro"))

print(f"""
[+]Exactitud:\t {100*(statistics.mean(lista_exactitud))} %
[+]Presicion:\t {100*(statistics.mean(lista_precison))} %
[+]F1-score:\t {100*(statistics.mean(lista_f1score))} %
[+]Sensibilidad: {100*(statistics.mean(lista_sensibilidad))} %
""")



[+]Exactitud:	 87.75 %
[+]Presicion:	 88.72451175227944 %
[+]F1-score:	 85.94971059261759 %
[+]Sensibilidad: 85.5014652014652 %



## Busqueda de mejores parametros

In [38]:
from sklearn.model_selection import GridSearchCV

# --> Parametros a evaluar
parametros = {'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]}

# --> Objeto de busqueda
busqueda = GridSearchCV(estimator=gnb, param_grid=parametros, scoring='accuracy', cv=10)

# --> Entrenamiento
busqueda = busqueda.fit(x, y)

# --> Resultados
print(f"[+]Mejor exactitud: {100*busqueda.best_score_} %")
print(f"[+]Mejores parametros: {busqueda.best_params_}")



[+]Mejor exactitud: 87.75 %
[+]Mejores parametros: {'var_smoothing': 1e-09}
